# Libraries Used

* ffmpeg-python
* av
* cmake
* dlib  (based on the python version)
* face-recognition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Celeb-DF\ v1/
%pwd

In [ ]:
%pip install ffmpeg-python
%pip install av
%pip install cmake
%pip install dlib
%pip install face-recognition

# **MESONET**

In [1]:
import av

import keras
from keras import layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam

import shutil
import imghdr
from PIL import Image

import os
import random

In [20]:
DS_ORG = './dataset_original/'
DS_IFRAME = './dataset_IFrames/'
DS_FACE = './dataset_face/'
DS_FINAL = './dataset_final/'
DS_SEG = './dataset_segments/'
DS_RAW = './dataset_raw/'
DS_RES = './dataset_residuals/'

MS_TRAIN = './dataset_mesonet/train_dataset/'
MS_TEST = './dataset_mesonet/test_dataset/'

CELEB_REAL = 'Celeb-real/'
CELEB_FAKE = 'Celeb-synthesis/'
YT_REAL = 'YouTube-real/'

REAL_VIDS = 'real_videos/'
FAKE_VIDS = 'fake_videos/'

SEG = ['seg_1_', 'seg_2_', 'seg_3_']

In [3]:
def create_model(input_size):
  model = keras.Sequential()

  model.add(layers.Conv2D(input_shape=input_size, filters=8, kernel_size=3, activation='relu', padding="same"))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2, 2, padding="same"))

  model.add(layers.Conv2D(input_shape=(128, 128, 8), filters=8, kernel_size=5, activation='relu', padding="same"))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2, 2, padding="same"))

  
  model.add(layers.Conv2D(input_shape=(64, 64, 8), filters=16, kernel_size=5, activation='relu', padding="same"))
  model.add(BatchNormalization())
  model.add(MaxPool2D(4, 4, padding="same"))

  
  model.add(layers.Conv2D(input_shape=(16, 16, 16), filters=16, kernel_size=5, activation='relu', padding="same"))
  model.add(BatchNormalization())
  model.add(MaxPool2D(4, 4, padding="same"))

  model.add(Flatten())

  model.add(Dropout(0.5))
  model.add(layers.Dense(16, activation='LeakyReLU'))

  model.add(Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  
  return model
  

In [4]:
input_size = (256, 256, 3)
model = create_model(input_size)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 8)       224       
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 8)      32        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 8)       1608      
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 8)      32        
 hNormalization)                                                 
                                                        

## DATASET CREATION

In [ ]:
# dir_real = 'dataset_REAL/'
# dir_fake = 'dataset_FAKE/'
# dir_train = './training_data/'
# dir_test = './testing_data/'

In [ ]:
# if os.path.exists(dir_train + dir_real) or os.path.exists(dir_train + dir_fake):
#   shutil.rmtree(dir_real)
#   shutil.rmtree(dir_fake)
# os.makedirs(dir_train + dir_real)
# os.makedirs(dir_train + dir_fake)
# os.makedirs(dir_test + dir_real)
# os.makedirs(dir_test + dir_fake)

In [25]:
def dataset_extract_frames(source_path, dest_path, vid, tag, count):
  frame_count = 0

  if(imghdr.what(os.path.join(source_path, vid)) == 'jpeg'):
    image = Image.open(source_path + vid)
    image.save(f'{dest_path}/vid_{tag}{count}_fr_{frame_count}.jpg')
    
    return

  vid = av.open(source_path + vid)
  for frame in vid.decode():
    image = frame.to_image()
    image.save(f'{dest_path}/vid_{tag}{count}_fr_{frame_count}.jpg')

    frame_count += 1

In [26]:
#extracting frames from Celeb-real face-cropped data
vid_count = 1
source_path = DS_FACE + CELEB_REAL
for video in os.listdir(source_path):
  # print(video, vid_count)
  dataset_extract_frames(source_path, MS_TRAIN + REAL_VIDS, video, 'cr', vid_count)
  vid_count += 1

In [27]:
#extracting frames from YouTube-real face-cropped data
vid_count = 1
source_path = DS_FACE + YT_REAL
for video in os.listdir(source_path):
  # print(video, vid_count)
  dataset_extract_frames(source_path, MS_TRAIN + REAL_VIDS, video, 'yr', vid_count)
  vid_count += 1

In [28]:
#extracting frames from Celeb-synthesis face-cropped data
vid_count = 1
source_path = DS_FACE + CELEB_FAKE
for video in os.listdir(source_path):
  # print(video, vid_count)
  dataset_extract_frames(source_path, MS_TRAIN + FAKE_VIDS, video, 'cf', vid_count)
  vid_count += 1

In [29]:
#extracting test data

def extract_test_data(source_path):
  frame_list = []
  for frame in os.listdir(source_path):
    frame_path = os.path.join(source_path, frame)
    frame_list.append(frame_path)

  size = int(20/100 * len(frame_list))
  sampled_list = random.sample(frame_list, size)
  return sampled_list

In [30]:
#extracting test data from real dataset

source_path = MS_TRAIN + REAL_VIDS
sampled_list = extract_test_data(source_path)
for frame_path in sampled_list:
  shutil.copy(frame_path, os.path.join(MS_TEST + REAL_VIDS, os.path.basename(frame_path)))
  os.remove(frame_path)

In [31]:
#extracting test data from fake dataset

source_path = MS_TRAIN + FAKE_VIDS
sampled_list = extract_test_data(source_path)
for frame_path in sampled_list:
  shutil.copy(frame_path, os.path.join(MS_TEST + FAKE_VIDS, os.path.basename(frame_path)))
  os.remove(frame_path)

## Keras Dataset Creation

In [32]:
#creating dataset from folders
def create_dataset(dir_path):
  ds = keras.utils.image_dataset_from_directory(
      directory = dir_path,
      labels = 'inferred',
      label_mode = 'binary',
      batch_size = 32,
      color_mode = 'rgb',
      shuffle = True,
      validation_split = 0.2,
      subset = 'validation',
      seed = 1
  )
  return ds

In [33]:
train_ds = keras.utils.image_dataset_from_directory(
      directory = MS_TRAIN,
      labels = 'inferred',
      label_mode = 'binary',
      batch_size = 32,
      color_mode = 'rgb',
      shuffle = True,
      validation_split = 0.2,
      subset = 'validation',
      seed = 1
  )
for data, labels in train_ds.take(1):
  print(data.shape)

Found 22627 files belonging to 2 classes.
Using 4525 files for validation.
(32, 256, 256, 3)


In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory=MS_TEST,
    labels='inferred',
    label_mode='binary'
)

for data, labels in test_ds.take(1):
  print(data.shape)